In [1]:
import cv2
import torch
import numpy as np
import time
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization
import torch
from torchvision import transforms
from PIL import Image
from datetime import datetime
import os
import glob

In [2]:
device =  torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Face Capturing

In [3]:
IMG_PATH = './data/test_images/'
count = 50 #Количество изображений каждого пользователя
# Ввести имя пользователя
usr_name = input("Input user name: ")
USR_PATH = os.path.join(IMG_PATH, usr_name)
leap = 1

MTCNN

In [4]:
# post_process=False так как нам не нужна нормализация при сохранении изображений
mtcnn = MTCNN(margin = 20, keep_all=False, select_largest = True, post_process=False, device = device)

Снимать 

In [5]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

while cap.isOpened() and count:
    isSuccess, frame = cap.read()
    if mtcnn(frame) is not None and leap%2:
        # Использовать datetime для сохранения информации изображений пользователя
        path = str(USR_PATH+'/{}.jpg'.format(str(datetime.now())[:-7].replace(":","-").replace(" ","-")+str(count)))
        face_img = mtcnn(frame, save_path = path)
        count-=1
    leap+=1
    cv2.imshow('Face Capturing', frame)
    if cv2.waitKey(1)&0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/ha/.local/lib/python3.10/site-packages/cv2/qt/plugins"


# Создать список изображений каждого пользователя

In [6]:
IMG_PATH = './data/test_images'
DATA_PATH = './data'

Нормализация в [-1, 1]

In [7]:
def trans(img):
    transform = transforms.Compose([
            transforms.ToTensor(),
            fixed_image_standardization
        ])
    return transform(img)

Обучение с использованием Pretrained models CASIA-Webface

In [ ]:
# classify = False чтобы получить Feature Vector в 512-мерном
model = InceptionResnetV1(
    classify=False,
    pretrained="casia-webface"
).to(device)

model.eval()

In [9]:
embeddings = []
names = []

for usr in os.listdir(IMG_PATH):
    embeds = []
    # Получить 50 Feature Vector из 50 изображений
    for file in glob.glob(os.path.join(IMG_PATH, usr)+'/*.jpg'):
        # print(usr)
        try:
            img = Image.open(file)
        except:
            continue
        with torch.no_grad():
            embeds.append(model(trans(img).to(device).unsqueeze(0))) 
    
    if len(embeds) == 0:
        continue

    # Нужен только один вектор из 50 векторов
    # Найти среднее значение каждой компонеты всех 50 векторов
    embedding = torch.cat(embeds).mean(0, keepdim=True) 
    
    # Добавить в список пользователей
    embeddings.append(embedding) 
    names.append(usr)
embeddings = torch.cat(embeddings) #[n,512]
names = np.array(names)

Сохранение

In [10]:
if device == 'cpu':
    torch.save(embeddings, DATA_PATH+"/faceslistCPU.pth")
else:
    torch.save(embeddings, DATA_PATH+"/faceslist.pth")
np.save(DATA_PATH+"/usernames", names)
print('Update Completed! There are {0} people in FaceLists'.format(names.shape[0]))

Update Completed! There are 1 people in FaceLists


# Face Recognition

In [11]:
frame_size = (640,480)
IMG_PATH = './data/test_images'
DATA_PATH = './data'

Функции

In [12]:
#Нормализация в [-1, 1]
def trans(img):
    transform = transforms.Compose([
            transforms.ToTensor(),
            fixed_image_standardization
        ])
    return transform(img)

#Получить список пользователей и Feature Vector
def load_faceslist():
    if device == 'cpu':
        embeds = torch.load(DATA_PATH+'/faceslistCPU.pth')
    else:
        embeds = torch.load(DATA_PATH+'/faceslist.pth')
    names = np.load(DATA_PATH+'/usernames.npy')
    return embeds, names

#Детектировать лицо и получить координаты ограничительной рамки
power = pow(10, 6)
def extract_face(box, img, margin=20):
    face_size = 160
    img_size = frame_size
    margin = [
        margin * (box[2] - box[0]) / (face_size - margin),
        margin * (box[3] - box[1]) / (face_size - margin),
    ]
    box = [
        int(max(box[0] - margin[0] / 2, 0)),
        int(max(box[1] - margin[1] / 2, 0)),
        int(min(box[2] + margin[0] / 2, img_size[0])),
        int(min(box[3] + margin[1] / 2, img_size[1])),
    ]
    img = img[box[1]:box[3], box[0]:box[2]]
    face = cv2.resize(img,(face_size, face_size), interpolation=cv2.INTER_AREA)
    face = Image.fromarray(face)
    return face

# Оценка классификации детектированного лица face - похожа на kNN (k =1)
def inference(model, face, local_embeds, threshold = 3):
    embeds = []
    embeds.append(model(trans(face).to(device).unsqueeze(0)))
    detect_embeds = torch.cat(embeds) 

    #Считать расстояния между вектором detect_embeds и каждым вектором из local_embeds
    #Получить матрицу расстояния [n,512]
    norm_diff = detect_embeds.unsqueeze(-1) - torch.transpose(local_embeds, 0, 1).unsqueeze(0)
    #Расстояние есть сумма квадратов каждой компоненты векторов
    norm_score = torch.sum(torch.pow(norm_diff, 2), dim=1) 
    
    min_dist, embed_idx = torch.min(norm_score, dim = 1)
    print(min_dist*power, names[embed_idx])
    
    if min_dist*power > threshold:
        return -1, -1
    else:
        return embed_idx, min_dist.double()
        

Тест

In [ ]:
# classify = False чтобы получить Feature Vector в 512-мерном
model = InceptionResnetV1(
    classify=False,
    pretrained="casia-webface"
).to(device)

model.eval()

MTCNN

In [14]:
mtcnn = MTCNN(thresholds= [0.7, 0.7, 0.8] ,keep_all=True, device = device)

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

prev_frame_time = 0
new_frame_time = 0


embeddings, names = load_faceslist()
while cap.isOpened():
    isSuccess, frame = cap.read()
    if isSuccess:
        boxes, _ = mtcnn.detect(frame)
        if boxes is not None:
            for box in boxes:
                bbox = list(map(int,box.tolist()))
                face = extract_face(bbox, frame)
                idx, score = inference(model, face, embeddings)
                
                # Показать bounding box
                if idx != -1:
                    frame = cv2.rectangle(frame, (bbox[0],bbox[1]), (bbox[2],bbox[3]), (0,0,255), 6)
                    score = torch.Tensor.cpu(score[0]).detach().numpy()*power
                    frame = cv2.putText(frame, names[idx] + '_{:.2f}'.format(score), (bbox[0],bbox[1]), cv2.FONT_HERSHEY_DUPLEX, 2, (0,255,0), 2, cv2.LINE_8)
                else:
                    frame = cv2.rectangle(frame, (bbox[0],bbox[1]), (bbox[2],bbox[3]), (0,0,255), 6)
                    frame = cv2.putText(frame,'Unknown', (bbox[0],bbox[1]), cv2.FONT_HERSHEY_DUPLEX, 2, (0,255,0), 2, cv2.LINE_8)
                
                # Показать 5 Точек в лице
                # if(not isinstance(points_list, list)):
                    # points_list= points_list.tolist()
                # for usr in points_list:
                    # for points in usr:
                        # frame = cv2.circle(frame, (int(points[0]), int(points[1])), radius=0, color=(0,0,255), thickness=10)
        
        
        new_frame_time = time.time()
        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        fps = str(int(fps))
        cv2.putText(frame, fps, (7, 70), cv2.FONT_HERSHEY_DUPLEX, 3, (100, 255, 0), 3, cv2.LINE_AA)

    cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(1)&0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
#Нажать ESC чтобы выключить
